In [1]:
import pandas as pd
import numpy as np
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, LpStatus, lpSum, value, getSolver
from pulp import LpStatus, LpStatusOptimal

In [2]:
locations_df = pd.read_csv("locations.csv")
routes_df = pd.read_csv("routes.csv")

In [3]:
supply_points_df = locations_df[locations_df['Location Type'].isin(['Hub', 'Focus City'])]
supply_points = dict(zip(supply_points_df['Location Name'], supply_points_df['Capacity']))


centers_df = locations_df[locations_df['Location Type'] == 'Center']
centers = dict(zip(centers_df['Location Name'], centers_df['Demand']))

centers = {city: float(demand.replace(",", "")) for city, demand in centers.items()}

costs = {}
for _, row in routes_df.iterrows():
    is_available = bool(row['IsAvailable?']) #Availability Constraint
    
    if is_available:
        try:
            source_name = locations_df.loc[locations_df['Location ID'] == row['Route Start'], 'Location Name'].values[0]
            dest_name = locations_df.loc[locations_df['Location ID'] == row['Route End'], 'Location Name'].values[0]
            costs[(source_name, dest_name)] = row['Price']
        except (IndexError, KeyError) as e:
            print(f"Error processing route from {row['Route Start']} to {row['Route End']}: {e}")

In [4]:
routes_df.head() # Data Inspection

,Route Start,Route End,Price,IsAvailable?
0,1,6,1.6,True
1,1,7,1.5,True
2,1,8,1.5,True
3,1,9,NaN,False
4,1,10,NaN,False


In [5]:
locations_df.head()

,Location ID,Location Name,Location Type,Capacity,Demand
0,1.0,Cincinnati/Northern Kentucky (CVG),Hub,95650.0,NaN
1,2.0,Alliance Fort Worth (AFW),Hub,44350.0,NaN
2,3.0,"Leipzig, Germany",Focus City,85000.0,NaN
3,4.0,"Hyderabad, India",Focus City,19000.0,NaN
4,5.0,"San Bernardino, CA, United States",Focus City,36000.0,NaN


In [6]:
model = LpProblem("Amazon_Cargo_Distribution", LpMinimize) #Method

# Decision Variables
x = {}
for D in supply_points:
    for FC in centers:
        if (D, FC) in costs:  
            x[D, FC] = LpVariable(f"ship_{D}_to_{FC}", lowBound=0, cat='Continuous')

# Objective Function
model += lpSum(costs[D, FC] * x[D, FC] for (D, FC) in x), "Total_Transportation_Cost"

In [7]:
#Demand Constraint
for FC in centers:
    available_routes = [D for D in supply_points if (D, FC) in x]
    if available_routes:
        model += lpSum(x[D, FC] for D in available_routes) == centers[FC], f"Demand_{FC}"
    else:
        print:(f"No Valid Routes to {FC}")


In [8]:
# Capacity Constraint
for D in supply_points:
    model += lpSum(x[D, FC] for FC in centers if (D, FC) in x) <= supply_points[D], f"Capacity_{D}"

In [9]:
#Non-Negativity Constraint

for (D, FC) in x:
    model += x[D, FC] >=0, f"NonNeg_{D}_to_{FC}"

In [10]:
solver = getSolver('PULP_CBC_CMD', msg=True)
model.solve(solver)

1

In [11]:
print(f"Model status: {LpStatus[model.status]}")

Model status: Optimal


In [12]:
if model.status == LpStatusOptimal:
    print("Solution found! Optimal distribution plan:")
    
    # Print shipment details
    for (D, FC) in sorted(x.keys()):
        if value(x[D, FC]) > 0:  # Only show non-zero shipments
            print(f"Ship {value(x[D, FC]):,.2f} tons from {D} to {FC} " 
                  f"at ${costs[D, FC]} per ton (total ${costs[D, FC] * value(x[D, FC]):,.2f})")
    
    # Print total cost
    print(f"\nTotal cost: ${value(model.objective):,.2f}")

Solution found! Optimal distribution plan:
Ship 975.00 tons from Alliance Fort Worth (AFW) to Austin at $0.25 per ton (total $243.75)
Ship 3,300.00 tons from Alliance Fort Worth (AFW) to Houston at $0.25 per ton (total $825.00)
Ship 1,100.00 tons from Alliance Fort Worth (AFW) to San Antonio at $0.25 per ton (total $275.00)
Ship 450.00 tons from Cincinnati/Northern Kentucky (CVG) to Albuquerque at $0.5 per ton (total $225.00)
Ship 420.00 tons from Cincinnati/Northern Kentucky (CVG) to Allentown at $0.5 per ton (total $210.00)
Ship 3,000.00 tons from Cincinnati/Northern Kentucky (CVG) to Atlanta at $0.5 per ton (total $1,500.00)
Ship 1,300.00 tons from Cincinnati/Northern Kentucky (CVG) to Baltimore at $0.5 per ton (total $650.00)
Ship 900.00 tons from Cincinnati/Northern Kentucky (CVG) to Charlotte at $0.5 per ton (total $450.00)
Ship 5,100.00 tons from Cincinnati/Northern Kentucky (CVG) to Chicago at $0.5 per ton (total $2,550.00)
Ship 3,300.00 tons from Cincinnati/Northern Kentucky (

In [13]:
# Demand Constraint Verification
print("\nDEMAND CONSTRAINT VERIFICATION:")
print("-------------------------------")
for D in centers:
    received = sum(value(x[FC, D]) for FC in supply_points if (FC, D) in x and value(x[FC, D]) > 0)
    required = centers[D]
    
    is_satisfied = abs(received - required) < 0.001 
    
    print(f"{D}: Received {received:,.2f} tons, Required {required:,.2f} tons - {'✓' if is_satisfied else '✗'}")


DEMAND CONSTRAINT VERIFICATION:
-------------------------------
Paris: Received 6,500.00 tons, Required 6,500.00 tons - ✓
Cologne: Received 640.00 tons, Required 640.00 tons - ✓
Hanover: Received 180.00 tons, Required 180.00 tons - ✓
Bangalore : Received 9,100.00 tons, Required 9,100.00 tons - ✓
Coimbatore: Received 570.00 tons, Required 570.00 tons - ✓
Delhi: Received 19,000.00 tons, Required 19,000.00 tons - ✓
Mumbai: Received 14,800.00 tons, Required 14,800.00 tons - ✓
Cagliari: Received 90.00 tons, Required 90.00 tons - ✓
Catania: Received 185.00 tons, Required 185.00 tons - ✓
Milan: Received 800.00 tons, Required 800.00 tons - ✓
Rome: Received 1,700.00 tons, Required 1,700.00 tons - ✓
Katowice: Received 170.00 tons, Required 170.00 tons - ✓
Barcelona: Received 2,800.00 tons, Required 2,800.00 tons - ✓
Madrid: Received 3,700.00 tons, Required 3,700.00 tons - ✓
Castle Donington: Received 30.00 tons, Required 30.00 tons - ✓
London: Received 6,700.00 tons, Required 6,700.00 tons - ✓


In [14]:
# Verify non-negativity constraints
print("\nNON-NEGATIVITY CONSTRAINT VERIFICATION:")
print("--------------------------------------")
negative_shipments = 0
for (FC, D) in x:
    shipment = value(x[FC, D])
    if shipment < 0:
        negative_shipments += 1
        print(f"Negative shipment: {FC} to {D} = {shipment:,.2f} tons")

if negative_shipments == 0:
    print("All shipments are non-negative ✓")


NON-NEGATIVITY CONSTRAINT VERIFICATION:
--------------------------------------
All shipments are non-negative ✓


In [15]:
# Verify route availability constraints - I built this into the way routes were created
print("\nROUTE AVAILABILITY VERIFICATION:")
print("-------------------------------")
unavailable_routes_used = 0
for (FC, D) in x:
    if value(x[FC, D]) > 0 and (FC, D) not in costs:
        unavailable_routes_used += 1
        print(f"Unavailable route used: {FC} to {D} = {value(x[FC, D]):,.2f} tons")

if unavailable_routes_used == 0:
    print("Only available routes were used ✓")


ROUTE AVAILABILITY VERIFICATION:
-------------------------------
Only available routes were used ✓


In [16]:
# Verify capacity constraints
print("\nCAPACITY CONSTRAINT VERIFICATION:")
print("---------------------------------")
for FC in supply_points:
    # Calculate total shipment from this supply point
    shipped = sum(value(x[FC, D]) for D in centers if (FC, D) in x and value(x[FC, D]) > 0)
    capacity = supply_points[FC]
    
    # Check if constraint is satisfied
    is_satisfied = shipped <= capacity + 0.001  # Small tolerance for floating-point errors
    
    print(f"{FC}: Shipped {shipped:,.2f} tons, Capacity {capacity:,.2f} tons - {'✓' if is_satisfied else '✗'}")


CAPACITY CONSTRAINT VERIFICATION:
---------------------------------
Cincinnati/Northern Kentucky (CVG): Shipped 60,615.00 tons, Capacity 95,650.00 tons - ✓
Alliance Fort Worth (AFW): Shipped 5,375.00 tons, Capacity 44,350.00 tons - ✓
Leipzig, Germany: Shipped 47,965.00 tons, Capacity 85,000.00 tons - ✓
Hyderabad, India: Shipped 19,000.00 tons, Capacity 19,000.00 tons - ✓
San Bernardino, CA, United States: Shipped 792.00 tons, Capacity 36,000.00 tons - ✓
